## Nettoyage de la base allocine

In [115]:
import pandas as pd 
import numpy as np 
from datetime import datetime
from dateutil import parser 

In [116]:
df_movies= pd.read_csv("df_film_ac.csv")

In [117]:
print(df_movies.dtypes)
#df_movies.isnull().sum()
#df_movies["date"].value_counts()

Titre original           object
Note press               object
Notes spectateur         object
Critiques presse         object
Critiques spectateurs    object
id allocine               int64
Box office               object
Budget                   object
date                     object
duree                    object
dtype: object


In [118]:
#modifier les types numériques 
df_movies['Box office'] = df_movies['Box office'].str.replace(' ', '').astype(float)
df_movies['Note press'] = df_movies['Note press'].str.replace(',', '.').astype(float)
df_movies['Notes spectateur'] = df_movies['Notes spectateur'].replace("--", np.nan)
df_movies['Notes spectateur'] = df_movies['Notes spectateur'].str.replace(',', '.').astype(float)

In [119]:
# Fonction pour convertir le mois en anglais
def french_to_english_month(month_french):
    months_mapping = {
        'janvier': 'January',
        'février': 'February',
        'mars': 'March',
        'avril': 'April',
        'mai': 'May',
        'juin': 'June',
        'juillet': 'July',
        'août': 'August',
        'septembre': 'September',
        'octobre': 'October',
        'novembre': 'November',
        'décembre': 'December'
    }
    return months_mapping.get(month_french.lower(), month_french)

# Remplacer les chaînes "nan" par des valeurs NaN
df_movies['date'] = df_movies['date'].replace('nan', np.nan)

# Appliquer la fonction pour convertir le mois en anglais
df_movies['date'] = df_movies['date'].apply(lambda x: ' '.join([french_to_english_month(word) for word in str(x).split()]) if pd.notna(x) else np.nan)

# Utiliser dateutil.parser.parse pour convertir les dates en objets datetime
df_movies['date'] = df_movies['date'].apply(lambda x: parser.parse(x, dayfirst=True) if isinstance(x, str) else x)



In [120]:
# Fonction pour convertir la durée en minutes
def convert_duration(duration_str):
    if isinstance(duration_str, str):
        # Supprimer les espaces et diviser la chaîne en parties
        parts = duration_str.replace(' ', '').split('h')

        # Vérifier la présence des heures et des minutes
        if len(parts) == 2:
            hours = int(parts[0])
            minutes = 0 if 'min' not in parts[1] else int(parts[1].replace('min', ''))
            
            # Calculer la durée totale en minutes
            total_minutes = hours * 60 + minutes
            
            return int(total_minutes)
    
    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return float('nan')

# Appliquer la fonction de conversion
df_movies['duree_minutes'] = df_movies['duree'].apply(convert_duration)


In [121]:
#Extraire le nombre de critique de la presse 

def extract_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Utiliser isdigit() pour extraire uniquement les chiffres
        return np.nan if not critiques_str.split()[0].isdigit() else int(critiques_str.split()[0])
    else:
        return np.nan

# Appliquer la fonction de conversion
df_movies['Nombre_de_critiques_presse'] = df_movies['Critiques presse'].apply(extract_critiques_count)


In [122]:
#critiques et votes spectateurs 

# Fonction pour extraire le nombre de notes et le nombre de critiques
def extract_notes_and_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Trouver les nombres dans la chaîne
        numbers = [int(word) for word in critiques_str.split() if word.isdigit()]

        # Extraire le nombre de notes et de critiques en fonction de la longueur de la liste "numbers"
        if len(numbers) == 1:
            return numbers[0], np.nan
        elif len(numbers) == 2:
            return numbers[0], numbers[1]

    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return np.nan, np.nan

# Appliquer la fonction de conversion
df_movies[['Nombre_de_notes_spectateurs', 'Nombre_de_critiques_spectateurs']] = df_movies['Critiques spectateurs'].apply(extract_notes_and_critiques_count).apply(pd.Series)

In [123]:
df_movies.head()

,Titre original,Note press,Notes spectateur,Critiques presse,Critiques spectateurs,id allocine,Box office,Budget,date,duree,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs
0,Shutter Island,3.8,4.4,31 critiques,80471 notes dont 4605 critiques,132039,127770000.0,80 000 000 $,2010-02-24,2h 17min,137.0,31.0,80471.0,4605.0
1,Inception,4.1,4.5,24 critiques,110095 notes dont 7212 critiques,143692,290948208.0,160 000 000 $,2010-07-21,2h 28min,148.0,24.0,110095.0,7212.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,20 critiques,52676 notes dont 2887 critiques,126693,291377000.0,150 000 000 $,2010-11-24,2h 26min,146.0,20.0,52676.0,2887.0
3,Prince of Persia: The Sands of Time,2.6,3.1,22 critiques,26730 notes dont 2133 critiques,126678,89981000.0,200 000 000 $,2010-05-26,2h 06min,126.0,22.0,26730.0,2133.0
4,The Book of Eli,2.4,3.3,20 critiques,10503 notes dont 1144 critiques,128955,92524000.0,80 000 000 $,2010-01-20,1h 49min,109.0,20.0,10503.0,1144.0


In [124]:
print(df_movies.dtypes)

Titre original                             object
Note press                                float64
Notes spectateur                          float64
Critiques presse                           object
Critiques spectateurs                      object
id allocine                                 int64
Box office                                float64
Budget                                     object
date                               datetime64[ns]
duree                                      object
duree_minutes                             float64
Nombre_de_critiques_presse                float64
Nombre_de_notes_spectateurs               float64
Nombre_de_critiques_spectateurs           float64
dtype: object


In [125]:
#on retire les colonnes que nous utilisons pas 
colonnes_a_retirer = ["Critiques presse","Critiques spectateurs","duree","Budget"]
df_movies_clean=df_movies.drop(colonnes_a_retirer,axis=1)

In [126]:
df_movies_clean.head()

,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs
0,Shutter Island,3.8,4.4,132039,127770000.0,2010-02-24,137.0,31.0,80471.0,4605.0
1,Inception,4.1,4.5,143692,290948208.0,2010-07-21,148.0,24.0,110095.0,7212.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,20.0,52676.0,2887.0
3,Prince of Persia: The Sands of Time,2.6,3.1,126678,89981000.0,2010-05-26,126.0,22.0,26730.0,2133.0
4,The Book of Eli,2.4,3.3,128955,92524000.0,2010-01-20,109.0,20.0,10503.0,1144.0


In [128]:
#retrait de tous les films sans critique spectateurs
#on garde que les films avec au moins plus de 5 commentaires

df_film_avec_critiques = df_movies_clean.dropna(subset=['Nombre_de_critiques_spectateurs'])
df_film_avec_critiques = df_film_avec_critiques[df_film_avec_critiques['Nombre_de_critiques_spectateurs'] >= 5]
df_film_avec_critiques


,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs
0,Shutter Island,3.8,4.4,132039,127770000.0,2010-02-24,137.0,31.0,80471.0,4605.0
1,Inception,4.1,4.5,143692,290948208.0,2010-07-21,148.0,24.0,110095.0,7212.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,20.0,52676.0,2887.0
3,Prince of Persia: The Sands of Time,2.6,3.1,126678,89981000.0,2010-05-26,126.0,22.0,26730.0,2133.0
4,The Book of Eli,2.4,3.3,128955,92524000.0,2010-01-20,109.0,20.0,10503.0,1144.0
...,...,...,...,...,...,...,...,...,...,...
11101,The Gateway,NaN,2.3,273584,NaN,2022-03-16,91.0,NaN,40.0,6.0
11102,"Batman: The Long Halloween, Part Two",NaN,3.7,292641,NaN,2021-08-04,84.0,NaN,114.0,7.0
11104,Mortal Kombat Legends: Battle of the Realms,NaN,3.3,294101,NaN,2021-09-15,80.0,NaN,50.0,6.0
11117,Black As Night,NaN,1.6,287632,NaN,2021-10-01,87.0,NaN,53.0,6.0


In [129]:
df_film_avec_critiques.to_csv('df_film_avec_critiques.csv', index=False)


## Nettoyage de la base TMDB

## Merging des deux bases